In [2]:
from pysparkling import *

In [3]:
import h2o
hc = H2OContext.getOrCreate(sc)

Connecting to H2O server at http://10.0.2.15:54321... successful.


H2O cluster uptime:,16 secs
H2O cluster version:,3.14.0.2
H2O cluster version age:,3 months and 10 days !!!
H2O cluster name:,sparkling-water-cloudera_local-1512245184552
H2O cluster total nodes:,1
H2O cluster free memory:,716 Mb
H2O cluster total cores:,1
H2O cluster allowed cores:,1
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://10.0.2.15:54321
H2O connection proxy:,None


In [4]:
hc.is_initialized

True

In [5]:
kkbox_csv = "/vagrant/project/final_with_churn.csv"  # modify this for your machine
# Alternatively, you can import the data directly from a URL
#loan_csv = "https://raw.githubusercontent.com/h2oai/app-consumer-loan/master/data/loan.csv"
data = h2o.import_file(kkbox_csv)  # 163,987 rows x 15 columns

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
data.shape

(992931, 20)

In [7]:
data.columns

[u'user_id',
 u'days_used',
 u'total_transactions',
 u'total_num_25',
 u'total_num_50',
 u'total_num_75',
 u'total_num_985',
 u'total_num_100',
 u'avg_unique_songs',
 u'avg_total_secs',
 u'avg_plan_length',
 u'avg_expected_plan_price',
 u'avg_actual_plan_price',
 u'max_expiration_date',
 u'min_transaction_date',
 u'city',
 u'age',
 u'gender',
 u'registered_via',
 u'is_churn']

In [8]:
#data['bad_loan'] = data['bad_loan'].asfactor()  #encode the binary repsonse as a factor
#data['bad_loan'].levels()  #optional: after encoding, this shows the two factor levels, '0' and '1'

data['is_churn'] = data['is_churn'].asfactor()
data['is_churn'].levels()

[['0', '1']]

In [9]:
data['min_transaction_date'].isna().unique()

C1
0


In [10]:
#clean up the data dropna
#for columns in total_num assume no usage for NAs
#example : df["sepal_len"] = (df["sepal_len"].isna()).ifelse(0, df["sepal_len"])
data['total_num_25'] = (data['total_num_25'].isna()).ifelse(0, data['total_num_25'])

In [11]:
columns = [u'total_num_50',
 u'total_num_75',
 u'total_num_985',
 u'total_num_100',
 u'avg_unique_songs',
 u'avg_total_secs',
 u'avg_plan_length',
 u'avg_expected_plan_price',
 u'avg_actual_plan_price']
for i in columns:
    data[i] = (data[i].isna()).ifelse(0, data[i])

In [12]:
data['gender'] = (data['gender'].entropy().isna() == 1).ifelse('NotProvided', data['gender'])

In [13]:
data['gender']

gender
\N
NotProvided
NotProvided
\N
male
\N
male
\N
NotProvided
NotProvided


In [14]:
data_cleaned = data.na_omit()
print data_cleaned.shape

(696634, 20)


In [ ]:
#raw_data['total_num_50'].isnull().value_counts()

In [15]:
# Partition data into 70%, 15%, 15% chunks
# Setting a seed will guarantee reproducibility

splits = data_cleaned.split_frame(ratios=[0.7], seed=1)  

train = splits[0]
test = splits[1]

In [16]:
print train.nrow
print test.nrow

487798
208836


Identify response and predictor variables

In [17]:
y = 'is_churn'
x = list(data.columns[1:len(data.columns) -1])

In [18]:
x

[u'days_used',
 u'total_transactions',
 u'total_num_25',
 u'total_num_50',
 u'total_num_75',
 u'total_num_985',
 u'total_num_100',
 u'avg_unique_songs',
 u'avg_total_secs',
 u'avg_plan_length',
 u'avg_expected_plan_price',
 u'avg_actual_plan_price',
 u'max_expiration_date',
 u'min_transaction_date',
 u'city',
 u'age',
 u'gender',
 u'registered_via']

In [19]:
import time
#initial time

In [20]:
# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [21]:
# Initialize the GLM estimator:
# Similar to R's glm() and H2O's R GLM, H2O's GLM has the "family" argument

glm_fit1 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit1')

In [22]:
t0 = time.time()
glm_fit1.train(x=x, y=y, training_frame=train)
glm_time = time.time() - t0
print ("logistic regression model fitted in %.3f s"
      % glm_time)

glm Model Build progress: |███████████████████████████████████████████████| 100%
logistic regression model fitted in 17.402 s


In [23]:
glm_perf1 = glm_fit1.model_performance(test)

In [24]:
print glm_perf1


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.0093796808413
RMSE: 0.0968487523993
LogLoss: 0.0453401148444
Null degrees of freedom: 208835
Residual degrees of freedom: 208816
Null deviance: 23524.7178989
Residual deviance: 20285.0117316
AIC: 20325.0117316
AUC: 0.865746651576
Gini: 0.731493303151
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.100121998648: 


,0,1,Error,Rate
0,204731.0,2002.0,0.0097,(2002.0/206733.0)
1,1552.0,551.0,0.738,(1552.0/2103.0)
Total,206283.0,2553.0,0.017,(3554.0/208836.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1001220,0.2366838,176.0
max f2,0.0742542,0.2586135,199.0
max f0point5,0.1259492,0.2293847,158.0
max accuracy,0.9988879,0.9899203,0.0
max precision,0.4761247,0.45,21.0
max recall,0.0001564,1.0,399.0
max specificity,0.9988879,0.9999903,0.0
max absolute_mcc,0.0987818,0.2293030,177.0
max min_per_class_accuracy,0.0100921,0.7799094,330.0
max mean_per_class_accuracy,0.0088990,0.7869973,336.0


Gains/Lift Table: Avg response rate:  1.01 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100031,0.1211405,22.6749341,22.6749341,0.2283389,0.2283389,0.2268188,0.2268188,2167.4934051,2167.4934051
,2,0.0200013,0.0658713,7.7046092,15.1915636,0.0775862,0.1529806,0.0770328,0.3038516,670.4609179,1419.1563610
,3,0.0300044,0.0477564,3.3750950,11.2521121,0.0339876,0.1133099,0.0337613,0.3376129,237.5095005,1025.2112139
,4,0.0400027,0.0373657,3.1864742,9.2361854,0.0320881,0.0930093,0.0318592,0.3694722,218.6474167,823.6185387
,5,0.0500010,0.0315530,2.7108810,7.9313745,0.0272989,0.0798698,0.0271041,0.3965763,171.0881007,693.1374475
,6,0.1000019,0.0188791,3.1288036,5.5300890,0.0315074,0.0556886,0.1564432,0.5530195,212.8803600,453.0089037
,7,0.1500029,0.0142194,2.2443698,4.4348493,0.0226010,0.0446594,0.1122206,0.6652401,124.4369756,343.4849277
,8,0.2000038,0.0110924,1.6357271,3.7350687,0.0164719,0.0376125,0.0817879,0.7470281,63.5727110,273.5068735
,9,0.3000010,0.0077235,1.1032176,2.8578130,0.0111095,0.0287785,0.1103186,0.8573466,10.3217621,185.7813036
,10,0.4000029,0.0056958,0.6989880,2.3181003,0.0070389,0.0233435,0.0699001,0.9272468,-30.1011962,131.8100326


In [25]:
print glm_perf1.auc()

0.865746651576


## Random Forest

import classifier

In [26]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator
# Import gridsearch
from h2o.grid.grid_search import H2OGridSearch

In [27]:
# Specify parameters for decision trees
hyper_parameters = {'ntrees': [200, 300, 400], 'max_depth': [10, 15, 20]}
#build grid estimator
rf = H2ORandomForestEstimator(model_id='rf_fit1',seed=6)
gs_rf = H2OGridSearch(rf,hyper_parameters)

In [28]:

rf_fit2 = H2ORandomForestEstimator(ntrees = 400, max_depth = 10, seed = 6, model_id='rf_fit2')
t0 = time.time()
rf_fit2.train(x=x, y=y, training_frame=train)

rf_time = time.time() - t0
print ("random forest model fitted in %.3f s"
      % rf_time)

drf Model Build progress: |███████████████████████████████████████████████| 100%
random forest model fitted in 416.951 s


In [29]:
rf_perf2 = rf_fit2.model_performance(test)

In [30]:
print rf_perf2


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.0084038621512
RMSE: 0.09167258124
LogLoss: 0.0380630509651
Mean Per-Class Error: 0.143807873419
AUC: 0.92289421605
Gini: 0.8457884321
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.13161910899: 


,0,1,Error,Rate
0,206217.0,516.0,0.0025,(516.0/206733.0)
1,1562.0,541.0,0.7427,(1562.0/2103.0)
Total,207779.0,1057.0,0.01,(2078.0/208836.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1316191,0.3424051,167.0
max f2,0.0534640,0.3152688,216.0
max f0point5,0.2201603,0.4396135,138.0
max accuracy,0.3338200,0.9907822,108.0
max precision,0.9255246,1.0,0.0
max recall,0.0002702,1.0,399.0
max specificity,0.9255246,1.0,0.0
max absolute_mcc,0.1371422,0.3590863,165.0
max min_per_class_accuracy,0.0188043,0.8396531,309.0
max mean_per_class_accuracy,0.0129892,0.8561921,329.0


Gains/Lift Table: Avg response rate:  1.01 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100031,0.0658917,30.8512205,30.8512205,0.3106750,0.3106750,0.3086068,0.3086068,2985.1220543,2985.1220543
,2,0.0200013,0.0493218,6.1351518,18.4961447,0.0617816,0.1862581,0.0613409,0.3699477,513.5151754,1749.6144739
,3,0.0300044,0.0420228,4.8962646,13.9621279,0.0493059,0.1406001,0.0489777,0.4189253,389.6264585,1296.2127880
,4,0.0400027,0.0385112,4.2803384,11.5422600,0.0431034,0.1162317,0.0427960,0.4617214,328.0338433,1054.2259988
,5,0.0500010,0.0365640,4.0901012,10.0521137,0.0411877,0.1012258,0.0408940,0.5026153,309.0101169,905.2113693
,6,0.1000019,0.0307593,3.1288036,6.5904586,0.0315074,0.0663666,0.1564432,0.6590585,212.8803600,559.0458647
,7,0.1500029,0.0220760,2.7959522,5.3256231,0.0281555,0.0536296,0.1398003,0.7988588,179.5952153,432.5623149
,8,0.2000038,0.0147690,1.9495585,4.4816070,0.0196323,0.0451302,0.0974798,0.8963386,94.9558474,348.1606980
,9,0.3000010,0.0080189,0.8036370,3.2556561,0.0080927,0.0327848,0.0803614,0.9767000,-19.6363026,225.5656115
,10,0.4000029,0.0025213,0.1378956,2.4762067,0.0013886,0.0249357,0.0137898,0.9904898,-86.2104401,147.6206656
